In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Set this to the gesture you are recording 
gesture_name = 'B'
output_dir = 'data'
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(0)
data = []

print("[INFO] Press 'r' to record frame, 'q' to quit and save.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            key = cv2.waitKey(10) & 0xFF
            if key == ord('r'):
                data.append(landmarks)
                print(f"[REC] {len(data)} frames saved")

    cv2.imshow("Capture", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save to CSV
df = pd.DataFrame(data)
df['label'] = gesture_name
df.to_csv(os.path.join(output_dir, f"{gesture_name}.csv"), index=False)
print(f"[DONE] Saved {len(data)} frames to {gesture_name}.csv")


[INFO] Press 'r' to record frame, 'q' to quit and save.
[REC] 1 frames saved
[REC] 2 frames saved
[DONE] Saved 2 frames to Y.csv
